In [1]:
import pickle
from collections import defaultdict
from framework.losses import interpoint_distance
import os
import pandas as pd

In [24]:
RESULTS_DIR = "./results/exp1"
N_RESULT_FILES = 10

def _load(name):
    with open(name, "rb") as f:
        return pickle.load(f)
K_VALUES = [5, 10, 20, 30, 40, 50, 60]
EPSILON_VALUES = [0.0000001, 0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1]

In [3]:
df  = pd.DataFrame(_load('./results/exp1/experimentone_1.pkl'))
df = df.drop(['emb_x', 'labels'], axis=1)
df

,size,sampling,dataset,algorithm,loss,convergence
0,0.3,random,mnist,umap,3.78816,scatter
1,0.3,random,mnist,umap,"[0.5476808905380334, 0.5862708719851577, 0.630...",nn_precision
2,0.3,random,mnist,umap,"[0.0018552875695732839, 0.014656771799628942, ...",fn_precision
3,0.3,random,mnist,umap,"([0.9896668399186258, 0.932879206666644, 0.745...",epsilon_precision_recall
4,0.3,random,mnist,tsne,0.100124,scatter
...,...,...,...,...,...,...
251,1.0,stratified,olivetti,umap,"([0.9975000000000019, 0.9975000000000019, 0.99...",epsilon_precision_recall
252,1.0,stratified,olivetti,tsne,0.0575142,scatter
253,1.0,stratified,olivetti,tsne,"[0.664, 0.54825, 0.489125, 0.47583333333333333...",nn_precision
254,1.0,stratified,olivetti,tsne,"[0.2165, 0.28875, 0.417625, 0.47725, 0.5188125...",fn_precision


In [66]:
df  = pd.DataFrame(_load('./results/exp1/experimentone_2.pkl'))
df = df.drop(['emb_x', 'labels'], axis=1)
df[df['convergence'] == 'nn_precision']

,size,sampling,dataset,algorithm,loss,convergence
1,0.3,random,mnist,umap,"[0.5476808905380334, 0.5862708719851577, 0.630...",nn_precision
5,0.3,random,mnist,tsne,"[0.5725417439703154, 0.5725417439703154, 0.607...",nn_precision
9,0.3,random,olivetti,umap,"[0.5583333333333333, 0.5483333333333333, 0.571...",nn_precision
13,0.3,random,olivetti,tsne,"[0.5666666666666667, 0.4975, 0.543333333333333...",nn_precision
17,0.3,stratified,mnist,umap,"[0.5317254174397031, 0.5834879406307978, 0.645...",nn_precision
...,...,...,...,...,...,...
237,1.0,random,olivetti,tsne,"[0.6555, 0.53125, 0.472375, 0.4695833333333333...",nn_precision
241,1.0,stratified,mnist,umap,"[0.45865331107401225, 0.4732888146911519, 0.51...",nn_precision
245,1.0,stratified,mnist,tsne,"[0.5523650528658876, 0.5220367278797997, 0.536...",nn_precision
249,1.0,stratified,olivetti,umap,"[0.6895, 0.617, 0.529375, 0.5066666666666667, ...",nn_precision


In [41]:
avg_epsilon_pr = df.copy()
avg_epsilon_pr = avg_epsilon_pr[avg_epsilon_pr['convergence'] == 'epsilon_precision_recall']
for entry in range(len(avg_epsilon_pr)):
        for j in range(len(EPSILON_VALUES)):
            avg_epsilon_pr['loss'].iloc[entry][0][j] = 0
            avg_epsilon_pr['loss'].iloc[entry][1][j] = 0


TypeError: 'tuple' object does not support item assignment

In [5]:
df  = pd.DataFrame(_load('./results/exp1/experimentone_3.pkl'))
df = df.drop(['emb_x', 'labels'], axis=1)
df[df['convergence'] == 'scatter']

,size,sampling,dataset,algorithm,loss,convergence
0,0.3,random,mnist,umap,3.78816,scatter
4,0.3,random,mnist,tsne,0.100124,scatter
8,0.3,random,olivetti,umap,10.3642,scatter
12,0.3,random,olivetti,tsne,0.569379,scatter
16,0.3,stratified,mnist,umap,4.75936,scatter
...,...,...,...,...,...,...
236,1.0,random,olivetti,tsne,0.0722516,scatter
240,1.0,stratified,mnist,umap,3.66906,scatter
244,1.0,stratified,mnist,tsne,0.0602525,scatter
248,1.0,stratified,olivetti,umap,7.04554,scatter


In [70]:
def create_avg_data(RESULTS_DIR, N_RESULT_FILES):
    print("Listing all result files...")
    for filename in os.listdir(RESULTS_DIR):
        print(filename)
    
    if len(os.listdir(RESULTS_DIR)) != N_RESULT_FILES:
        print("Some runs are missing please check the results folder")
    
    print("---------------------------------------------------------")
    
    print("Combining files now....")
    exp2_runs = {}
    i=0
    print("Loading Data...")
    for run in os.listdir(RESULTS_DIR):
        result = _load(os.path.join(RESULTS_DIR, run))
        result = pd.DataFrame(result)
        result = result.drop(['emb_x', 'labels'], axis=1)
        
        exp2_runs[i] = result
        i +=1
        
    # ---------------------------------------------------------------------------------------- 
    
    # Scatter Table
    avg_scatter = exp2_runs[0].copy()
    avg_scatter = avg_scatter[avg_scatter['convergence'] == 'scatter']
    avg_scatter['loss'] = 0
    
    #Epsilon Precision-Recall Table:
    avg_epsilon_pr = exp2_runs[0].copy()
    avg_epsilon_pr = avg_epsilon_pr[avg_epsilon_pr['convergence'] == 'epsilon_precision_recall']
    for entry in range(len(avg_epsilon_pr)):
        for j in range(len(EPSILON_VALUES)):
            avg_epsilon_pr['loss'].iloc[entry][0][j] = 0
            avg_epsilon_pr['loss'].iloc[entry][1][j] = 0
            
    #NN and FN Precision Table
    avg_nn_pr = exp2_runs[0].copy()
    avg_nn_pr = avg_nn_pr[avg_nn_pr['convergence'] == 'nn_precision']
    avg_fn_pr = exp2_runs[0].copy()
    avg_fn_pr = avg_fn_pr[avg_fn_pr['convergence'] == 'fn_precision']
    
    for entry in range(len(avg_nn_pr)):
        for j in range(len(K_VALUES)):
            avg_nn_pr['loss'].iloc[entry][j] = 0
            avg_fn_pr['loss'].iloc[entry][j] = 0
    
    
    # Summing up
    for ind in range(N_RESULT_FILES):
        df = exp2_runs[ind]
        
        # Combine Scatter:
        avg_scatter['loss'] += df[df['convergence'] == 'scatter']['loss']
        
        # Combine Epsilon Precision Recall:
        df_ep = df[df['convergence'] == 'epsilon_precision_recall']
        
        for entry in range(len(avg_epsilon_pr)):
            for j in range(len(EPSILON_VALUES)):
                avg_epsilon_pr['loss'].iloc[entry][0][j] += df_ep['loss'].iloc[entry][0][j]
                avg_epsilon_pr['loss'].iloc[entry][1][j] += df_ep['loss'].iloc[entry][1][j]
                
        # Combine NN and FN Precision:
        df_nn = df[df['convergence'] == 'nn_precision']
        df_fn = df[df['convergence'] == 'fn_precision']
        
        for entry in range(len(avg_nn_pr)):
            for j in range(len(K_VALUES)):
                avg_nn_pr['loss'].iloc[entry][j] += df_nn['loss'].iloc[entry][j]
                avg_fn_pr['loss'].iloc[entry][j] += df_fn['loss'].iloc[entry][j]
    
    #Scaling down to get average:
    avg_scatter['loss'] = avg_scatter['loss'] / N_RESULT_FILES
    
    for entry in range(len(avg_epsilon_pr)):
        for j in range(len(EPSILON_VALUES)):
            avg_epsilon_pr['loss'].iloc[entry][0][j] = avg_epsilon_pr['loss'].iloc[entry][0][j]/N_RESULT_FILES
            avg_epsilon_pr['loss'].iloc[entry][1][j] = avg_epsilon_pr['loss'].iloc[entry][1][j]/N_RESULT_FILES
            
    for entry in range(len(avg_nn_pr)):
            for j in range(len(K_VALUES)):
                avg_nn_pr['loss'].iloc[entry][j] = avg_nn_pr['loss'].iloc[entry][j]/N_RESULT_FILES
                avg_fn_pr['loss'].iloc[entry][j] = avg_fn_pr['loss'].iloc[entry][j]/N_RESULT_FILES
    

    
    
        
    return avg_scatter, avg_epsilon_pr, avg_nn_pr, avg_fn_pr
        
        

In [71]:
t,s,u,v = create_avg_data(RESULTS_DIR, N_RESULT_FILES)

Listing all result files...
experimentone_9.pkl
experimentone_8.pkl
experimentone_10.pkl
experimentone_5.pkl
experimentone_4.pkl
experimentone_6.pkl
experimentone_7.pkl
experimentone_3.pkl
experimentone_2.pkl
experimentone_1.pkl
---------------------------------------------------------
Combining files now....
Loading Data...


In [74]:
t.to_csv('exp1_scatter.csv')
s.to_csv('exp1_epsilon_pr.csv')
u.to_csv('exp1_nn_pr.csv')
v.to_csv('exp1_fn_pr.csv')